In [1]:
import sys; sys.path.insert(0,"../..")
from agd import AutomaticDifferentiation as ad
import numpy as np
import cupy as cp

In [6]:
delta = ad.Dense.identity(constant=cp.array((1,1)))
delta = delta.reshape((2,1))

In [7]:
delta.concatenate((delta[0,:],delta[1,:]))

denseAD(array([1, 1]),
array([[1, 0],
       [0, 1]]))

In [ ]:
delta.stack([0,delta[1]])

In [4]:
print(l)

[0, denseAD(array(1),array([0, 1]))]


In [ ]:
ad.ad_generic.array(l)

In [ ]:
ad.array([delta[0],0.2+delta[1]])

In [3]:
delta[0]

hi - array 1


denseAD(array(1),array([1, 0]))

In [4]:
delta[1]

hi - array 1


denseAD(array(1),array([0, 1]))

In [5]:
l=[delta[0],delta[1]]

hi - array 1
hi - array 1


In [6]:
ad.array([0,1])

hi - array [0, 1]
hi - array 0
hi - array 1
hi -stack


array([0, 1])